In [1]:
%load_ext autoreload
%autoreload 2
import sys, os
sys.path.append(os.path.abspath("../../"))
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from src.config import Config, load_spreadsheet
from src.rag import (
    ChromaSlideStore,
    HyperbolicScorer,
    MinScorer,
    PresentationRetriever,
    ScorerTypes,
)
from src.eval.eval_mlflow import EvaluationConfig, RAGEvaluator

import mlflow

In [3]:
# Mlflow setup logging
mlflow.langchain.autolog()

# Setup components
project_config = Config()
llm = project_config.model_config.load_vsegpt(model="openai/gpt-4o-mini")
embeddings = project_config.embedding_config.load_vsegpt()

storage = ChromaSlideStore(collection_name="pres0", embedding_model=embeddings)

db_path = project_config.navigator.processed / "eval" / "mlruns.db"
eval_config = EvaluationConfig(
    experiment_name="PresRetrieve_mlflow",
    metrics=["presentation_match", "page_match"],
    scorers=[MinScorer(), HyperbolicScorer()],
    tracking_uri=f"sqlite:////{db_path}",
)

evaluator = RAGEvaluator(
    storage=storage,
    config=eval_config,
    # llm=llm
)

In [4]:
from IPython.display import display
# Load questions
sheet_id = os.environ["BENCHMARK_SPREADSHEET_ID"]
df = evaluator.load_questions_from_sheet(sheet_id)

df_eval = df.sample(5)
display(df_eval)

# Run evaluation
evaluator.run_evaluation(df_eval)



,pres_name,question,page,content,comment,MinScorer,Hyperbolic Scorer
7,Kept_Подвижной состав РФ_2024 (20 стр),Развитие парка цистерн,14,text,NaN,NaN,NaN
23,SP_Навигатор_по_мерам_гос_поддержки_2024_74_стр,Презентация в которой рассматривались субсидии...,23,text,NaN,PASS,PASS
8,Kept_Подвижной состав РФ_2024 (20 стр),Про что рассказывал Сергей Казачков?,"2,20",text,NaN,PASS,PASS
21,4.Эволюция отбора кандидатов в системе товарны...,Презентация про маркетплейсы,,general,NaN,FAIL,FAIL
4,ЯиП_Энергетический_переход_Вызовы_и_возможност...,В какой презентации говорили о снижении доли и...,14,text,NaN,NaN,NaN


MlflowException: Invalid artifact path: '/home/groot/Desktop/projects/global/PresRAG/presentation-rag/data/processed/eval/detailed_results.csv'. Names may be treated as files in certain cases, and must not resolve to other names when treated as such. This name would resolve to '/home/groot/Desktop/projects/global/PresRAG/presentation-rag/data/processed/eval/detailed_results.csv'